# **Hyperparameter Tuning:** Advanced

### **Top HPT Techniques:**
| Technique | Library |
------------|----------
| GridSearchCV | Scikit-Learn |
| RandomSearchCV | Scikit-Learn |
| HalvingGridSearchCV | Scikit-Learn |
| HalvingRandomSearchCV | Scikit-Learn |
| Bayesian Optimization | bayes_opt |
| Bayesian Optimization | HyperOPT |
| Bayesian Optimization | Scikit-Optimize |
| Automate Hyperparameter Tuning | Optuna |
| Automate Hyperparameter Tuning | FLAML |
| Genetic Algorithms | TPOT |

## Importing Generic Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier

## Dataset that'll be used throughout is " **Iris** "

In [2]:
df = sns.load_dataset("iris")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
X = df.iloc[:, :2]
y = df.iloc[:, -1]

## Model Evaluation using Cross Validation

In [4]:
from sklearn.model_selection import cross_val_score
KNN = KNeighborsClassifier()
cv = cross_val_score(KNN, X, y, cv=10, scoring="accuracy")
np.mean(cv)

0.76

---
## **Model Hyperparameters**: KNN

In [5]:
# Parameters for KNN Classifier
para_dict = {
    "n_neighbors": [3,5,10,15],
    "weights": ["uniform", "distance"],
    "algorithm": ["ball_tree", "kd_tree", "brute"],
    "leaf_size": [20,30,40,50]
}

print(para_dict)

{'n_neighbors': [3, 5, 10, 15], 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree', 'brute'], 'leaf_size': [20, 30, 40, 50]}


---
## Model Tuning using **GridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV

KNN = KNeighborsClassifier()
KNN_grid = GridSearchCV(estimator = KNN, 
                       param_grid = para_dict, 
                       cv = 5, 
                    #    verbose=2, 
                       n_jobs = -1)

KNN_grid.fit(X,y)

print(f"BEST PARAMETERS ARE: \n{KNN_grid.best_params_}")
print(f"\nBEST SCORE IS: {KNN_grid.best_score_}")

BEST PARAMETERS ARE: 
{'algorithm': 'ball_tree', 'leaf_size': 20, 'n_neighbors': 15, 'weights': 'uniform'}

BEST SCORE IS: 0.7933333333333333


---
## Model Tuning using **RandomSearchCV**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

KNN = KNeighborsClassifier()
KNN_grid = RandomizedSearchCV(estimator = KNN, 
                       param_distributions = para_dict, 
                       cv = 5, 
                    #    verbose=2, 
                       n_jobs = -1)

KNN_grid.fit(X,y)

print(f"BEST PARAMETERS ARE: \n{KNN_grid.best_params_}")
print(f"\nBEST SCORE IS: {KNN_grid.best_score_}")

BEST PARAMETERS ARE: 
{'weights': 'uniform', 'n_neighbors': 10, 'leaf_size': 40, 'algorithm': 'brute'}

BEST SCORE IS: 0.78


---
## Model Tuning using **Halving GridSearchCV**
Halving Techniques are experimental so in order to enable them we should use<br>
`from sklearn.experimental import enable_halving_search_cv`

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

KNN = KNeighborsClassifier()
KNN_grid = HalvingGridSearchCV(estimator = KNN, 
                                param_grid = para_dict, 
                                cv = 5, 
                                #    verbose=2, 
                                n_jobs = -1)

KNN_grid.fit(X,y)

print(f"BEST PARAMETERS ARE: \n{KNN_grid.best_params_}")
print(f"\nBEST SCORE IS: {KNN_grid.best_score_}")

BEST PARAMETERS ARE: 
{'algorithm': 'ball_tree', 'leaf_size': 20, 'n_neighbors': 15, 'weights': 'uniform'}

BEST SCORE IS: 0.7888888888888889


---
## Model Tuning using **Halving RandomSearchCV**
Halving Techniques are experimental so in order to enable them we should use<br>
`from sklearn.experimental import enable_halving_search_cv`

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

KNN = KNeighborsClassifier()
KNN_grid = HalvingRandomSearchCV(estimator = KNN, 
                       param_distributions = para_dict, 
                       cv = 5, 
                    #    verbose=2, 
                       n_jobs = -1)

KNN_grid.fit(X,y)

print(f"BEST PARAMETERS ARE: \n{KNN_grid.best_params_}")
print(f"\nBEST SCORE IS: {KNN_grid.best_score_}")

BEST PARAMETERS ARE: 
{'weights': 'distance', 'n_neighbors': 15, 'leaf_size': 40, 'algorithm': 'kd_tree'}

BEST SCORE IS: 0.8222222222222222


---
## Model Tuning using **bayes_opt** (Bayesian Optimization)
Bayesian Optimization evaluates the past model information to select hyperparameter values to build the newer model.<br>
To install it use<br>
`$ pip install bayesian-optimization`
### In order to use it you must have:
- **Parameters:** A dictionary where each parameter is defined
- **Black-Box:** A function that binds the algorithm with the parameters

In [ ]:
# Parameters for KNN Classifier. Unable to make it work for non-integer parameters
parameters = {
    "n_neighbors": [10,15],
    # "weights": ["uniform", "distance"],
    # "algorithm": ["ball_tree", "kd_tree", "brute"],
    "leaf_size": [40,50]
}

print(parameters)

{'n_neighbors': [10, 15], 'leaf_size': [40, 50]}


In [ ]:
def black_box(n_neighbors, leaf_size):
  para = {}
  parameters['n_neighbors'] = round(n_neighbors)
  parameters['leaf_size'] = round(leaf_size)

  score = cross_val_score(KNeighborsClassifier(**para), 
                          X, y, cv=5, scoring="accuracy").mean()

  return score

In [ ]:
from bayes_opt import BayesianOptimization

KNN_bo = BayesianOptimization(black_box, parameters, random_state=42)
KNN_bo.maximize(init_points = 10, n_iter = 4)

print(f"BEST PARAMETERS ARE: \n{KNN_bo.max['params']}")
print(f"\n BEST SCORE IS: {KNN_bo.max['target'] }")

|   iter    |  target   | leaf_size | n_neig... |
-------------------------------------------------
| 1         | 0.7667    | 43.75     | 14.75     |
| 2         | 0.7667    | 47.32     | 12.99     |
| 3         | 0.7667    | 41.56     | 10.78     |
| 4         | 0.7667    | 40.58     | 14.33     |
| 5         | 0.7667    | 46.01     | 13.54     |
| 6         | 0.7667    | 40.21     | 14.85     |
| 7         | 0.7667    | 48.32     | 11.06     |
| 8         | 0.7667    | 41.82     | 10.92     |
| 9         | 0.7667    | 43.04     | 12.62     |
| 10        | 0.7667    | 44.32     | 11.46     |
| 11        | 0.7667    | 49.98     | 10.02     |
| 12        | 0.7667    | 48.28     | 11.5      |
| 13        | 0.7667    | 49.96     | 14.95     |
| 14        | 0.7667    | 40.06     | 12.32     |
BEST PARAMETERS ARE: 
{'leaf_size': 43.74540118847362, 'n_neighbors': 14.75357153204958}

 BEST SCORE IS: 0.7666666666666666


---
## Model Tuning using **HyperOPT** (Bayesian Optimization)
To install it use<br>
`pip install hyperopt`
### In order to use it you must have:
- **Space:** A dictionary where each parameter is defined by HyperOPT standards
- **Objective:** A function that binds the algorithm with the space
- **Trial:** A method that'll be used in `fmin` to test our the algorithm
- **fmin:** A function that'll try to minimize the loss value

In [ ]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [ ]:
space = {'n_neighbors': hp.choice('n_neighbors', [3,5,10,15]),
        'weights': hp.choice('weights', ["uniform", "distance"]),
        'algorithm': hp.choice('algorithm', ["ball_tree", "kd_tree", "brute"]),
        'leaf_size': hp.choice('leaf_size', [20,30,40,50])
        }

In [ ]:
def objective(space):
  model = KNeighborsClassifier(n_neighbors = space['n_neighbors'],
                               weights = space['weights'],
                               algorithm = space['algorithm'],
                               leaf_size = space['leaf_size']
                              )

  accuracy = cross_val_score(model, X, y, cv = 5).mean()

  # We aim to maximize accuracy, therefore we return it as a negative value
  return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
from sklearn.model_selection import cross_val_score
from hyperopt.fmin import space_eval
trials = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 80,
            rstate=np.random.RandomState(42),
            trials= trials)

space_eval(space, best)
print(f"BEST PARAMETERS ARE: \n{space_eval(space, best)}")
print(f"\n BEST SCORE IS: {- trials.best_trial['result']['loss']}")

100%|██████████| 80/80 [00:02<00:00, 37.25it/s, best loss: -0.7933333333333333]
BEST PARAMETERS ARE: 
{'algorithm': 'kd_tree', 'leaf_size': 50, 'n_neighbors': 15, 'weights': 'uniform'}

 BEST SCORE IS: 0.7933333333333333


---
## Model Tuning using **Scikit-Optimize** (Bayesian Optimization)
To install it use<br>
`pip install scikit-optimize`
### In order to use it you must have:
- **Search Space:** A dictionary where each parameter is defined by HyperOPT standards
- **Surrogate:** A model that binds the algorithm with the space

In [ ]:
# Parameters for KNN Classifier
search_space = {
    "n_neighbors": [3,5,10,15],
    "weights": ["uniform", "distance"],
    "algorithm": ["ball_tree", "kd_tree", "brute"],
    "leaf_size": [20,30,40,50]
}

print(search_space)

{'n_neighbors': [3, 5, 10, 15], 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree', 'brute'], 'leaf_size': [20, 30, 40, 50]}


In [ ]:
from skopt import BayesSearchCV

KNN_surrogate = BayesSearchCV(estimator = KNeighborsClassifier(), 
                       search_spaces = search_space, n_jobs=-1, cv=5)
KNN_surrogate.fit(X, y)

print(f"BEST PARAMETERS ARE: \n{KNN_surrogate.best_params_}")
print(f"\n BEST SCORE IS: {KNN_surrogate.best_score_}")

BEST PARAMETERS ARE: 
{'algorithm': 'ball_tree', 'leaf_size': 30, 'n_neighbors': 15, 'weights': 'uniform'}

BEST SCORE IS: 0.7933333333333333


---
## Model Tuning using **Optuna** (Automatic Hyperparameter Tuning)
To install it use<br>
`pip install optuna`

### In order to use it you must have:
- **Study:** An objective function with hyperparameters and models in it
- **Trials:** A tral is a single execution of the study. You need multiple trials to achive better results

In [ ]:
import sklearn.neighbors

def objective(trial):
  classifier = trial.suggest_categorical('classifier','KNeighbors')
  
  # Parameters for KNN Classifier
  n_neighbors = trial.suggest_int('n_neighbors', 3,15,2)
  weights = trial.suggest_categorical('weights', ["uniform", "distance"])
  algorithm = trial.suggest_categorical('algorithm', ["ball_tree", "kd_tree", "brute"])
  leaf_size = trial.suggest_int('leaf_size', 5,50,5)
      
  clf = sklearn.neighbors.KNeighborsClassifier(
      n_neighbors = n_neighbors, weights = weights,
      algorithm = algorithm, leaf_size = leaf_size
  )
  
  score = cross_val_score(clf,X_train,y_train, n_jobs=-1, cv=3).mean()

  return score

In [ ]:
import optuna

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best = study.best_trial

print(f"BEST PARAMETERS ARE: \n{best.params}")
print(f"\nBEST SCORE IS: {best.value}")

BEST PARAMETERS ARE: 
{'classifier': 'h', 'n_neighbors': 11, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 45}

BEST SCORE IS: 0.7946894262683736


---
## Model Tuning using **FLAML** (Automatic Hyperparameter Tuning)
To install it use<br>
`pip install flaml`
`pip install flaml[notebook]`

In [10]:
from flaml import AutoML
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

automl_settings = {
    "time_budget": 60,  # Seconds
    "metric": 'accuracy', # Evaluation Metric
    "estimator_list": ["kneighbor"], # Estimator
    "task": 'classification' # Supervised ML Task
}
autoML = AutoML()
autoML.fit(X_train, y_train, **automl_settings)

print(f"BEST PARAMETERS ARE: \n{autoML.model.estimator}")
print(f"\nBEST SCORE IS: {autoML.score(X_test, y_test)}")

BEST PARAMETERS ARE: 
KNeighborsClassifier(n_jobs=-1, n_neighbors=55, weights='distance')

BEST SCORE IS: 0.7894736842105263


---
## Model Tuning using **TPOT** (Genetric Algorithms)
To install it use<br>
`pip install tpot`
### How it works?:
- **Train Models:** First we train multiple models, let's say 20 models
- **Select Best:** Select half best models. That'd be 10 best models out of 20
- **Offsprings:** Create similar models (say 15) of the best ones using their Hyperparameters
- **Generations:** Repeat the entire process to build multiple generations (say 3) of models. The original models would be the great-grandfathers of these new models
- **Survival:** In this way the best model would survive at the end
### In order to use it you must have:
- **TensorFlow:** You need TensorFlow library for TPOT to work

In [ ]:
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

df = load_iris()
X = df.data[:, 0:2]
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 0, early_stop= 12,
                                 config_dict={'sklearn.neighbors.KNeighborsClassifier': para_dict}, 
                                 cv = 5, scoring = 'accuracy')
tpot_output = tpot_classifier.fit(X_train,y_train)


print(f"BEST PARAMETERS ARE: \n{tpot_output.fitted_pipeline_[1]}")
print(f"\nBEST SCORE IS: {tpot_output.score(X_test, y_test)}")

BEST PARAMETERS ARE: 
KNeighborsClassifier(algorithm='ball_tree', leaf_size=50, n_neighbors=3)

BEST SCORE IS: 0.7105263157894737
